In [105]:
# import libraries

import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns
sns.set()

In [106]:
# parameters
path_for_stock_list = 'inputFiles/finviz_penny.csv'
test_date = '2023-01-01'
#start_loadint_date
VolRise = 0.99        # open position, after current volume will be more than VolRise value
AvgVolDayLimit = 10   # average for shifting average
ExitAfter = 0.25      # close position after price achieve ExitAfter
StopLoss = -0.07      # close position after price meets stop
RiskPertrade = 2000   # buy for sum 
TradeAfterDays = 3    # begin trade after TradeAfterDays days from start
price_filter = 0.5     # below this price we open pos
price_Stock_filter = 5  # above this price we don't load stock
high_prevClose = 0.1 # if high more than previous close on high_prevClose
UpperBandVol = 0.7      # do not use if price is in range of  UpperBandVol and LowwerBandVol - please take a volume
LowwerBandVol = 0.05    #   if price is in range of  UpperBandVol and LowwerBandVol - please take a volume
positionValue = 10000   # the sum you need to collect
openPosStatus = False 
days_test = 360 # 180 by default #360 for all stocks
days_close = 340 # 160 by default #340 for all stocks
days_of_trade_period = 330 # 150 by default #330 for all stocks
volume_potensial =0 
close_signal = 2       # 1 = 100% growth, 2 = growth to $1 price, 3 = hold position for 1 month, 4 = after achieving 100% growth, close when price falls below SMA5
hold_loss_pos_days = 20 # if position holds 20 days and it's lossing - please close trade


In [107]:
#read stocks for trading
list_stocks = pd.read_csv(path_for_stock_list, index_col="Ticker")

#filtering stocks
list_stocks = list_stocks[list_stocks['Industry']!='Shell Companies']
list_stocks = list_stocks.loc[:,['Company','Industry','Sector','Country']]
list_stocks['NotDate'] = test_date



In [108]:

# дата старту набору позиції
# дата закінчення набору позиції
class StockBalance():
    positionValue = 10000
   
    def __init__(self, ticker):
        self.ticker = ticker
        self.market_value = 0
        self.date_end_pos = 0 # last date when we collect stock
        self.qty = 0
        self.avg_price = 0
        self.volume_pot = 0
        self.need = positionValue
        

    
    def add(self,amount, price, posdate):
        self.qty += amount
        val=price*amount
        self.market_value += val
        self.avg_price = self.market_value/self.qty
        self.date_end_pos = posdate

    
    def is_full(self):
        return self.market_value >=StockBalance.positionValue
    
    def clear(self):
        self.qty = 0
        self.market_value = 0
        self.avg_price = 0
    
    def volume_potensial(self, high, low, close, volume):
        
        if abs(high-low)==0:
            self.volume_pot
        else:
            self.volume_pot = ((abs(close-low)/abs(high-low))*volume)*0.05
        if self.volume_pot == 0 :
            self.volume_pot = volume*0.05
        
    def isclosesignal(self, nom_of_sig, price, cur_date, smaprice):
        if self.market_value>0:  # if there is open position
            
            if nom_of_sig == 1:
                if (price > self.avg_price*2):
                    return True
            if nom_of_sig == 2:
                if self.avg_price>1:
                    return True
            if nom_of_sig == 3:
                if (cur_date-self.date_end_pos)> datetime.timedelta(20): 
                    return True
            if nom_of_sig == 4:
                if (price > self.avg_price*2) & (price < smaprice):
                    return True
        else:
            return False
      
    def stoptrade(self,curentdate,lastdate):
        return False
        if self.market_value>0:
        # check is it time to close position
            if (curentdate-datefromlist)>=datetime.timedelta(days_close):
                return True
            
            else:
                return False
            
           
        
        
        

def good_price(ticker_name,date_from_list):
    
    # function to calculate appropriate price
    res = False
    try:
        stock_ticker = yf.Ticker(ticker_name)

        r = stock_ticker.history(max,"1d",date_from_list)
        price = r.iloc[0]["Close"]
        if (price<price_Stock_filter) & (price > LowwerBandVol):
            res = True
    except:
        res = False

    return res



In [109]:

# load ticker list from excel-file

all_res = []
data_group=[]
data_group_res=[]

stock_res =[]
stock_small_res=[]
#len(list_stocks)

# here begin to iterate between stocks
for i in range(len(list_stocks)): # потім прописати весь список
    symbol = list_stocks.index[i] # беремо акцію по індексу

    datefromlist = test_date
   
    
    # error it may be situation, when price is fall down 
    if good_price(symbol,datefromlist)==False: # if price meet conditions
        continue
    
    # this is blok where stock suit out coundition
    mystock = StockBalance(symbol)
    openPosStatus = False
    # to calc average volume use 20 days before
    end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    start_date_dt = end_date_dt - datetime.timedelta(days=19)
    start_date = start_date_dt.strftime('%Y-%m-%d') # convert back to string

   
    
    #load stock price
    
    try:
        wdf = yf.download(symbol, start_date, end_date)
    except ValueError:
        print(f"Error: {symbol} is not a valid symbol.")
        continue
        exit(1)
        
    
    print(symbol)
    #wdf = tick.history(start=start_date, end= end_date)
    #tick = yf.Ticker(symbol)


    # building features in working dataframe
    wdf['avgvol']= wdf['Volume'].rolling(window=AvgVolDayLimit, win_type=None).mean()  # середній обєм за 10 днів
    wdf['VolRise'] =wdf['Volume']/wdf['avgvol']-1      # building vol signal
    wdf['SMA5']= wdf['Close'].rolling(5).mean()
    wdf['PrevClose'] = wdf.Close.shift(1)      # write previous close
    wdf['volsignal']=wdf['VolRise']>VolRise            # volume signal column
    wdf['GreenCandleSig'] = wdf['Close']>wdf['Open']   # green candle
    wdf['HighGrowPrevClose'] = (wdf["High"]/wdf['PrevClose']-1)> high_prevClose # high is bigger than previous low
    #wdf['TradeAfter']=wdf.index
    #wdf['TradeAfterDaysSig'] = wdf['TradeAfter'] > datefromlist + datetime.timedelta(days=TradeAfterDays)
    wdf['OpenPrice']=0
    wdf['openPosStatus']=False
    wdf['VolumePoten']=0
    wdf['MarketValue']=0
    wdf['AvgPrice']=0
    wdf['Profit']=0
    wdf['test']=""
    wdf['StopPrice']=0
    wdf['TargetPrice']=0
    wdf['Ticker']=symbol
    
    wdf = wdf.iloc[-2:]
    lastrow=wdf.tail(1).index # last row in order to stop calculations
    

    # main module
    PriceOpen = 0
    StopPrice = 0
    TPPrice = 0
    CloseSignal = 0
    status = 0
    
    
    for index, row in wdf.iterrows():  # change to itertuple....
        
        # calculate volume potential
        mystock.volume_potensial(row['High'],row['Low'],row['Close'],row['Volume'])
        wdf.at[index,'VolumePoten']= mystock.volume_pot    
        
        # looking for opening signal 
        if (mystock.market_value ==0): #  if we haven't opened position and we have 150 days 
            
            if row['volsignal'] & row['GreenCandleSig'] & row['HighGrowPrevClose']:  # if Volume > than AvgVolume and green candle and high more then 
                if row["Close"]>price_filter: # if price for opening is bigger than limit - please skip it
                    continue

                # YES we find signal day
                openPosStatus = True  # there is signal 
                wdf.at[index,'openPosStatus'] = openPosStatus # record status in dataframe
                status=1
                
                # volume size block
                
        # the next day after signal appear we check condition in order to open position
        # if all conditions suit - we continue to collect positions
        
        if (status ==0) & (openPosStatus == True) & (mystock.is_full()==False) :
            #
            # calc how many money we need to collect
            # how much capital we need
            need_cap = positionValue - mystock.market_value 
            mid_price = (row['Close']+row['Open'])/2 # take avg.price for position
            
            if need_cap >=0:
                if (mid_price * wdf.at[index,'VolumePoten']) >= need_cap: # if potensial suit our demand
                    size = need_cap/mid_price
                    mystock.add(size, mid_price, index)
                    wdf.at[index,'Profit']= (size*mid_price)*-1
                    
                    
                else: 
                    
                    size = wdf.at[index,'VolumePoten']
                    mystock.add(size, mid_price, index)
                    wdf.at[index,'Profit']= (size*mid_price)*-1
            
            # record MarketValue
            wdf.at[index,'MarketValue']= mystock.market_value
            wdf.at[index,'AvgPrice']= mystock.avg_price
            
            
        if mystock.is_full()==True: # position is full we waiting for close signal
            openPosStatus = False
            
    #     # if price rise to my goal, please close position
        if (mystock.isclosesignal(close_signal, row['Close'], index, row['SMA5']) == True):
            # ready to close position
            mid_price = (row['Close']+row['Open'])/2 # take avg.price for position
            
            if wdf.at[index,'VolumePoten'] > mystock.qty: # close full position
                wdf.at[index,'Profit']= mystock.qty*mid_price
                mystock.clear() # clear market value, qty, avg.price
                openPosStatus = False
            else:
                wdf.at[index,'Profit']= wdf.at[index,'VolumePoten'] * mid_price # take all avaible stocks
                mystock.qty=(mystock.qty-wdf.at[index,'VolumePoten']) # decrease stock count 
                openPosStatus = False
        
        # there is condition for closing 
        if mystock.stoptrade(index,lastrow[0])==True:
            mid_price = (row['Close']+row['Open'])/2 # take avg.price for position
            
            if wdf.at[index,'VolumePoten'] > mystock.qty: # close full position
                wdf.at[index,'Profit']= mystock.qty*mid_price
                mystock.clear() # clear market value, qty, avg.price
                openPosStatus = False
            else:
                wdf.at[index,'Profit']= wdf.at[index,'VolumePoten'] * mid_price # take all avaible stocks
                mystock.qty=(mystock.qty-wdf.at[index,'VolumePoten']) # decrease stock count 
                openPosStatus = False
            
            
        
#         

        status = 0
        #wdf.at[index,'test']=lastrow[0]

        #if there is open position and its last row we close all pos
        if (lastrow[0]==index) & (mystock.market_value>0) :
                        
            mid_price = (row['Close']+row['Open'])/2 # take avg.price for position
            wdf.at[index,'Profit']= mystock.qty * mid_price
            
            openPosStatus=False
            #wdf.at[index,'test']=lastrow[0]
        
        # log block 
        # mylog=('mid_price', mid_price,'qty=', mystock.qty, 'avg_price', mystock.avg_price, 'mark_value=',mystock.market_value,'lastRow=', lastrow[0])
        # wdf.at[index,'test']=mylog
             
    #тут закінчується блок перебору строк акції і йде на наступну акцію
    # forcing close
    #mid_price = (row['Close']+row['Open'])/2 # take avg.price for position
    #wdf.at[index,'Profit']= mystock.qty * mid_price
    
   
    
    #siteDay_CurDate = str(index-datefromlist)
    #ProfitTrades = (wdf['Profit']>0).sum()
    #LossTrades = (wdf['Profit']<0).sum()
    #WinRate = round(ProfitTrades/tradecount,1)
    #AvgHold = round((wdf['OpenPrice']!=0).sum()/tradecount,1)
    stock_small_res=[symbol,round(wdf['Profit'].sum(),2)]
    
    data_group=wdf['Profit']
    stock_res.append(stock_small_res)
    data_group_res.append(data_group)
    # блок запису dataFrame:
    all_res.append(wdf)
      
    df_output = pd.concat(all_res)
    
    df_data =pd.concat(data_group_res)
    



    
      
        


[*********************100%***********************]  1 of 1 completed
AAU
[*********************100%***********************]  1 of 1 completed
ABVC


C:\Users\Vova\AppData\Local\Temp\ipykernel_1724\4020342697.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdf.at[index,'VolumePoten']= mystock.volume_pot


[*********************100%***********************]  1 of 1 completed
ACB


C:\Users\Vova\AppData\Local\Temp\ipykernel_1724\4020342697.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdf.at[index,'VolumePoten']= mystock.volume_pot


[*********************100%***********************]  1 of 1 completed
ACON


...

C:\Users\Vova\AppData\Local\Temp\ipykernel_1724\4020342697.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdf.at[index,'VolumePoten']= mystock.volume_pot


[*********************100%***********************]  1 of 1 completed
ZOM


C:\Users\Vova\AppData\Local\Temp\ipykernel_1724\4020342697.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdf.at[index,'VolumePoten']= mystock.volume_pot


[*********************100%***********************]  1 of 1 completed
ZYNE


C:\Users\Vova\AppData\Local\Temp\ipykernel_1724\4020342697.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdf.at[index,'VolumePoten']= mystock.volume_pot


In [110]:

stock_df = pd.DataFrame(stock_res, columns = ['Stock','Profit'])
stock_df = stock_df.loc[stock_df['Profit']!=0]

#stock_df.loc['Total'] = stock_df.sum(numeric_only=True) # add Total substring


df_day = pd.DataFrame(df_data )
df_day.index.name ="Date"
df_day = df_day.loc[df_day['Profit']!=0] # remove 0 values
df_day = df_day.groupby([df_day.index.date]).sum() # групуємо по даті
df_day.sort_index()


#total analysis
#total, avg, countTrades, profitCount, lossCount
#tot_analysis = [stock_df['Profit'].sum(), round(stock_df['Profit'].mean()), len(stock_df['Profit']), len(stock_df[stock_df['Profit']>0]),len(stock_df[stock_df['Profit']<0])]
anal_df = pd.DataFrame(tot_analysis, index = ['total','avg profit','countTrades','profitCount','lossCount'])


with pd.ExcelWriter("outputFiles/output.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_output.to_excel(writer, sheet_name="full_B")
    df_day.to_excel(writer, sheet_name="by date")
    stock_df.to_excel(writer, sheet_name="statByStock")
    anal_df.to_excel(writer, sheet_name="TotalStat")
    

In [111]:
 wdf = wdf.iloc[-2:]

In [112]:
wdf

,Open,High,Low,Close,Adj Close,Volume,avgvol,VolRise,SMA5,PrevClose,...,OpenPrice,openPosStatus,VolumePoten,MarketValue,AvgPrice,Profit,test,StopPrice,TargetPrice,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-02,0.48,0.493,0.454,0.489,0.489,286400,247080.0,0.159139,0.4906,0.463,...,0,False,12851.279245,0,0,0,,0,0,ZYNE
2023-03-03,0.47,0.496,0.464,0.496,0.496,249700,245120.0,0.018685,0.4858,0.489,...,0,False,12485.000000,0,0,0,,0,0,ZYNE


In [113]:
import pandas as pd

# create a sample DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve'],
    'Age': [25, 30, 35, 40, 45],
    'Gender': ['F', 'M', 'M', 'M', 'F']
}
df = pd.DataFrame(data)

# keep only the last 2 rows
#df = df.iloc[-2:]

# display the result
print(df)

      Name  Age Gender
0    Alice   25      F
1      Bob   30      M
2  Charlie   35      M
3     Dave   40      M
4      Eve   45      F


In [114]:
df = df.iloc[-2:0]

In [115]:
df

,Name,Age,Gender
